In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
#from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import neighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble

In [3]:
df = pd.read_csv('csv_files/atp_data.csv')

<center><h1><span style="color: #26B260"> Les algorithmes </span> </h1></center>

In [4]:
KNN = neighbors.KNeighborsClassifier(n_neighbors=3, metric='minkowski', weights='distance')
Decision_tree = DecisionTreeClassifier(criterion = 'entropy')
Random_forest = ensemble.RandomForestClassifier(criterion='entropy', max_depth = 30, n_estimators = 80)

<center><h1> <span style="color: #26B260">  La fonction de prédiction pour un match </span> </h1></center>

In [5]:
def model(row, status,needed_data, classifier):
    list_of_predection = []       # la liste des prediction 1: pour un joueur qui gagne un match et 
                                  # 0: pour un jour qui perd un match
    
    list_of_resultat = []         # liste des resultats du bookmaker
    list_of_row = []              # liste des lignes sélectionné pour faire l'analyse
    
    player = df.loc[row][status]  # On selectione un joueur "winner" ou "Loser"
    
    df1 = df.iloc[0:row+1]        # On selectionne un sous dataset qui contient toutes les informations du dataset
                                  # de 0 à row
 
                                   
    
    # On prend du df1 seulment les observations ou le joueur a joué 
    df_player = df1[(df1['Winner'] == player) | (df1['Loser'] == player)].sort_values(by=['Date'], ascending= True)
    
    # on enlève la vairiables inutiles
    df_player = df_player.drop(['Date','Location', 'Comment','ATP', 'PSW', 'PSL', 'B365W', 'B365L', 
                                'Round', 'Wsets', 'Lsets'], axis = 1)
    
    # On crée une colonne qui remplace le winner el le loser de chaque match par 1 et 0 respectivement.
    df_player['Winner'] = df_player['Winner'].replace(to_replace = player, value = 1)
    df_player['Loser'] = df_player['Loser'].replace(to_replace = player, value = 0)
    df_player['result'] = df_player.apply(lambda row: row['Winner'] if type(row['Winner']) == int 
                                                           else row['Loser'], axis = 1)
    
    
    # On supprime les colonnes Winner et loser qui ne sont plus utiles pour la suite
    df_player = df_player.drop(['Winner', 'Loser'], axis = 1)
    df_player = df_player.dropna(axis = 0, how = 'any')
    
    # On separe la variables cible des variables explicatives
    target = df_player['result']
    data = df_player.drop('result', axis = 1)
    
    # on décode les variables qualitatives par la fonction get_dummies
    data = data.join(pd.get_dummies(data[['Tournament','Series', 'Court', 'Surface']]))
    
    # On supprime les variables qualitatives        
    data = data.drop(['Tournament','Series', 'Court', 'Surface'], axis = 1)
    
    
    
    if len(data) > needed_data:         # condition qu'un joueur à jouer un certain nombre de match = needed_data 
                                        # pour lancer l'algorithme
        list_of_row.append(row)
        slice = int(len(data)*0.75)     # on séléctionne 75% des informations comme ensemble d'entrainement
        
          # Cette boucle fonctionne de la manière suivante:
          # 1- à la premiere iteration on lance l'agorithme sur l'ensemble d'entrainement indiqué en haut 
          #    et on fait une prediction sur la ligne nombre = slice + 1
          # 2- puis on ajoute la ligne "slice + 1" à l'ensemble d'entrainement et on relance l'algorithme 
          #    sur cet ensemble pour prédire le résultat du joueur sur la ligne slice + 2
          # 3- on repete les memes étapes pour trouver une prediction sur un ensemble des observation de 
          #    longueur [slice: len(data) - 1] qui sera considére dans la suite comme ensemble de test et 
          #    de trouver aussi un prédiction du resultat du joueur sur la derniere ligne du data
        for i in range(slice, len(data)):
            
            X_train = data.iloc[0:i]
            y_train = target.iloc[0:i]
            X_test = data.iloc[i:i+1]
            y_test = np.array(target.iloc[i:i+1])

            clf = classifier
            clf.fit(X_train, y_train)
            
            y_pred = clf.predict(X_test)
            list_of_predection.append(y_pred[0])
            list_of_resultat.append(y_test)

    else:
        return f'Not enough data for the {status} player {player}, Where the available informations are {len(data)} lines'
    
        
    list=[]    # cette liste contient les bon résultats de l'algorithme

    for value1, value2 in zip(list_of_predection,list_of_resultat):
        if value1 == value2:
            list.append(f'good predection')
        elif value1 != value2:
            list.append(f'bad predection')
            
    y_pred = clf.predict(data.tail(1))
    
    z = pd.DataFrame(list).value_counts(normalize = True)
    
    if y_pred[0] == 0:
        return 'the fidelity of the algorithm is:',z,f'The program expects that the player {player} will lose the match and the prediction based on {len(data)} lines', 'lose', list_of_row
    else:
        return 'the fidelity of the algorithm is:',z,f'The program expects that the player {player} will win the match and the prediction based on {len(data)} lines', 'win', list_of_row

<center><h2><span style="color: #26B260"> Prédiction d'un joueur sur un match donné </span> </h2></center> 

In [6]:
model(row = 32400, status = 'Winner', needed_data = 50, classifier = KNN)

('the fidelity of the algorithm is:',
 good predection    0.810345
 bad predection     0.189655
 dtype: float64,
 'The program expects that the player Haas T. will win the match and the prediction based on 463 lines',
 'win',
 [32400])

<center><h2><span style="color: #26B260"> Lancement d'un modèle de classification sur plusieurs observations </span> </h2></center> 

In [7]:
list_pred = []
list_bet365 = []
list_of_row = []

status_to_predict = 'Winner'
for row in range(35000,44700):
    z = model(row = row, status = status_to_predict, needed_data = 50, classifier = KNN)
    list_pred.append(z[3])
    #list_bet365.append(z[4])
    if isinstance(z[4][0], int):
        list_of_row.append(z[4][0])

KeyboardInterrupt: 

<center><h2><span style="color: #26B260"> Collection des bons résultats cumulés dans une liste </span> </h2></center> 

In [ ]:
list_pred_final = []
for item in list_pred:
    if item == 'win':
        list_pred_final.append('win_pred')
    elif item == 'lose':
        list_pred_final.append('lose_pred')
        
        
list_pred_binary = []
list_pred_opp = []
for i in range(len(list_pred_final)):
    if status_to_predict == 'Winner':
        if list_pred_final[i] == 'win_pred':
            list_pred_binary.append(1)
            list_pred_opp.append(1)
        elif list_pred_final[i] == 'lose_pred':
            list_pred_binary.append(0)
            list_pred_opp.append(-1)
            
    elif status_to_predict == 'Loser':
        if list_pred_final[i] == 'win_pred':
            list_pred_binary.append(0)
            list_pred_opp.append(-1)
        elif list_pred_final[i] == 'lose_pred':
            list_pred_binary.append(1)
            list_pred_opp.append(1)

l1=[list_pred_binary[0]]
l2=[list_pred_opp[0]]

for i in range(1,len(list_pred_final)):
    value1 = list_pred_binary[i] + l1[i-1]
    value2 = list_pred_opp[i] + l2[i-1]
    l1.append(value1)
    l2.append(value2)

<center><h2><span style="color: #26B260"> affichage du taux de précision de l'algorithme et du bookmaker </span> </h2></center> 

In [ ]:
list_of_bookmaker = []
list_bookmaker_opp = []

resultat_dataframe = pd.DataFrame()

resultat_dataframe.index = list_of_row

resultat_dataframe['algorithm_prediction'] = list_pred_binary
#resultat_dataframe['liste de bookmakers'] = list_bet365_final
resultat_dataframe['B365W'] = df.iloc[resultat_dataframe.index]['B365W']
resultat_dataframe['B365L'] = df.iloc[resultat_dataframe.index]['B365L']

for rate1, rate2 in zip(resultat_dataframe['B365W'], resultat_dataframe['B365L']):
    if rate1 < rate2:
        list_of_bookmaker.append(1)
        list_bookmaker_opp.append(1)
      
    elif rate1 > rate2:
        list_of_bookmaker.append(0)
        list_bookmaker_opp.append(-1)
        
    else:
        list_of_bookmaker.append(1)
        list_bookmaker_opp.append(1)
        
resultat_dataframe['bookmaker_prediction'] = list_of_bookmaker

display(resultat_dataframe.sample(30))

print("la prediction de l'algorithme est:")
display(resultat_dataframe['algorithm_prediction'].value_counts(normalize = True))

print("\nla prediction du bookmaker est:")
display(resultat_dataframe['bookmaker_prediction'].value_counts(normalize = True))

resultat_dataframe.shape

<center><h2><span style="color: #26B260"> visualisation des résultats sur un graphe </span> </h2></center> 

In [ ]:

L1=[list_of_bookmaker[0]]
L2=[list_bookmaker_opp[0]]

    
for i in range(1,len(list_bookmaker_opp)):
    value1 = list_of_bookmaker[i] + L1[i-1]
    value2 = list_bookmaker_opp[i] + L2[i-1]
    L1.append(value1)
    L2.append(value2)   


plt.figure(figsize=(15,13))
plt.subplot(2,1,1)
plt.plot([i for i in range(0, len(resultat_dataframe))], l1, 'y', label="Algo résultat")
plt.plot([i for i in range(0, len(resultat_dataframe))], L1, 'b', label="Bookmaker résultat")
plt.xlabel("Nombre d'observations")
plt.ylabel('Résultat Cumulé')
plt.title('Bonne_pred: 1, Mauvaise_pred: 0')
plt.legend();


plt.subplot(2,1,2)
plt.plot([i for i in range(0, len(resultat_dataframe))], l2, 'y', label="Algo résultat")
plt.plot([i for i in range(0, len(resultat_dataframe))], L2, 'b', label="Bookmaker résultat")
plt.xlabel("Nombre d'observations")
plt.ylabel('Résultat Cumulé')
plt.title('Bonne_pred: 1, Mauvaise_pred: -1')
plt.legend();


<center><h2><span style="color: #26B260"> Résultat Final </span> </h2></center> 

In [ ]:
win = resultat_dataframe[resultat_dataframe['algorithm_prediction'] == 1]


lose = resultat_dataframe[resultat_dataframe['algorithm_prediction'] == 0]

print(f"Le montant total de gain sur {len(L1)} matchs en déposant un euro sur chaque match prévue gagnant par l'algorithme est :{round(win['B365W'].sum() - len(win) - len(lose))} euros")

win_book = resultat_dataframe[resultat_dataframe['bookmaker_prediction'] == 1]

lose_book = resultat_dataframe[resultat_dataframe['bookmaker_prediction'] == 0]

print(f"Le montant total de gain sur {len(L1)} matchs en déposant un euro sur chaque match prévue gagnant par le bookmaker est :{round(win_book['B365W'].sum() - len(win_book) - len(lose_book))} euros")